Install Dependencies

In [1]:
!pip install --quiet langchain InstructorEmbedding sentence_transformers faiss-gpu pypdf langchain-community langchain_chroma wikipedia langchainhub langchain_huggingface

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 973.5/973.5 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.7/224.7 kB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.4/290.4 kB 34.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 69.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.2/310.2 kB 37.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.4/124.4 kB 16.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 526.8/526.8 kB 46.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 kB 12.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 86.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.4/62.4 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━

Import Libraries

In [3]:
import os
from huggingface_hub import notebook_login  ##setting up Hugging face Interface using hf tokens


from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper  ##Wikipedia as a Interface for 1st tool

from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceBgeEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter  ##creating Retriver

from langchain.tools.retriever import create_retriever_tool #converting retriever to retrievar tool {2nd tool}

from langchain import HuggingFaceHub
from langchain_huggingface import ChatHuggingFace  #to connect llm models from huggingface & convert them to chathugging face model in Langchain framework

from langchain import hub #for loading prompts

from langchain.agents import create_tool_calling_agent
from langchain.agents import AgentExecutor  #create agents from tool & then create a agent executor for executing queries


from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory  #enabling memory for the agents

Setup Hugging face Login

In [ ]:
os.environ['HUGGINGFACEHUB_API_TOKEN'] = ''

In [38]:
notebook_login()

Create Wikipedia as first tool. This is available as Default tool from Langchain library

In [4]:
api_wrapper=WikipediaAPIWrapper(top_k_results=1,doc_content_chars_max=1000)
wiki=WikipediaQueryRun(api_wrapper=api_wrapper)

In [5]:
wiki('tell me about machine learning')

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `BaseTool.__call__` was deprecated in langchain-core 0.1.47 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


'Page: Explainable artificial intelligence\nSummary: Explainable AI (XAI), often overlapping with Interpretable AI, or Explainable Machine Learning (XML), either refers to an artificial intelligence (AI) system over which it is possible for humans to retain intellectual oversight, or refers to the methods to achieve this. The main focus is usually on the reasoning behind the decisions or predictions made by the AI which are made more understandable and transparent. XAI counters the "black box" tendency of machine learning, where even the AI\'s designers cannot explain why it arrived at a specific decision.\nXAI hopes to help users of AI-powered systems perform more effectively by improving their understanding of how those systems reason. XAI may be an implementation of the social right to explanation. Even if there is no such legal right or regulatory requirement, XAI can improve the user experience of a product or service by helping end users trust that the AI is making good decisions

Create a website retriever as 2nd Tool

In [ ]:
loader = WebBaseLoader("https://docs.smith.langchain.com/overview")
docs = loader.load()
documents = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=200
).split_documents(docs)



model_name = "BAAI/bge-large-en"
model_kwargs = {"device": "cuda"}
encode_kwargs = {"normalize_embeddings": True}
hf_embedding = HuggingFaceBgeEmbeddings(
    model_name=model_name, model_kwargs=model_kwargs, encode_kwargs=encode_kwargs
)

vector = FAISS.from_documents(documents, hf_embedding)
retriever = vector.as_retriever()

In [7]:
retriever.invoke("how to upload a dataset")[0]

Document(page_content='description="A sample dataset in LangSmith.")client.create_examples(    inputs=[        {"postfix": "to LangSmith"},        {"postfix": "to Evaluations in LangSmith"},    ],    outputs=[        {"output": "Welcome to LangSmith"},        {"output": "Welcome to Evaluations in LangSmith"},    ],    dataset_id=dataset.id,)# Define your evaluatordef exact_match(run, example):    return {"score": run.outputs["output"] == example.outputs["output"]}experiment_results = evaluate(    lambda input: "Welcome " + input[\'postfix\'], # Your AI system goes here    data=dataset_name, # The data to predict and grade over    evaluators=[exact_match], # The evaluators to score the results    experiment_prefix="sample-experiment", # The name of the experiment    metadata={      "version": "1.0.0",      "revision_id": "beta"    },)import { Client, Run, Example } from "langsmith";import { evaluate } from "langsmith/evaluation";import { EvaluationResult } from "langsmith/evaluation";co

In [9]:
#once we initiate retriever using create_retriever_tool function we can convert the retriever to retriever tool
retriever_tool = create_retriever_tool(
    retriever,
    "langsmith_search",
    "Search for information about LangSmith. For any questions about LangSmith, you must use this tool!",
)

List of Tools we have created

In [10]:
tools = [wiki, retriever_tool]

Initiate Open Source LLM Model from Hugging face

In [60]:
llm=HuggingFaceHub(repo_id="meta-llama/Meta-Llama-3-8B-Instruct",
                                        model_kwargs={"max_new_tokens": 1000,
    "return_full_text": False,
    "temperature": 0.2,
    "do_sample": False,
                                                      })  #more model parameters can be added here

In [61]:
# convert the llm loaded from Hugging face to chathuggingface model which is supported in Langchain.
#this step is done to enable bind method which binds llm with tools.
#this helps in selecting a specific tool - or no tool, or multiple tools!
#This is useful when you want to allow the LLM to have more flexibility in how it should respond

llm_lc = ChatHuggingFace(llm=llm)

tokenizer_config.json:   0%|          | 0.00/51.0k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [62]:
llm_with_tools = llm_lc.bind_tools(tools)

Creating a Prompt

In [55]:
# Get the prompt to use - you can modify this!
prompt = hub.pull("hwchase17/openai-functions-agent")  #using langchain hub we can pull most relevant prompts available in the library
prompt.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are a helpful assistant')),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}')),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

Creating Agents with help llm, tools & prompt we have Initiated in above steps

In [63]:
agent = create_tool_calling_agent(llm_with_tools, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

Lets test our Agent Executor

In [65]:
agent_executor.invoke({"input": "hi!"})



> Entering new AgentExecutor chain...
Hi there! It's nice to meet you! Is there something I can help you with or would you like to chat? I'm all ears!

> Finished chain.


{'input': 'hi!',
 'output': "Hi there! It's nice to meet you! Is there something I can help you with or would you like to chat? I'm all ears!"}

In [66]:
agent_executor.invoke({"input": "how can langsmith help with testing?"})



> Entering new AgentExecutor chain...
Langsmith is a powerful tool that can significantly help with testing in several ways:

1. **Automated Testing**: Langsmith allows you to write automated tests for your language models, which can help you catch errors and inconsistencies in your model's output. You can write tests to verify the accuracy of your model's responses, and Langsmith will run these tests automatically, providing you with detailed reports on the results.

2. **Test Data Generation**: Langsmith can generate test data for your language models, which can be used to train and evaluate your models. This can be especially helpful when you're working with large datasets and need to generate a diverse set of test cases.

3. **Model Evaluation**: Langsmith provides a range of evaluation metrics that can be used to assess the performance of your language models. These metrics can help you identify areas where your model needs improvement and provide insights into how to optimize i

{'input': 'how can langsmith help with testing?',
 'output': "Langsmith is a powerful tool that can significantly help with testing in several ways:\n\n1. **Automated Testing**: Langsmith allows you to write automated tests for your language models, which can help you catch errors and inconsistencies in your model's output. You can write tests to verify the accuracy of your model's responses, and Langsmith will run these tests automatically, providing you with detailed reports on the results.\n\n2. **Test Data Generation**: Langsmith can generate test data for your language models, which can be used to train and evaluate your models. This can be especially helpful when you're working with large datasets and need to generate a diverse set of test cases.\n\n3. **Model Evaluation**: Langsmith provides a range of evaluation metrics that can be used to assess the performance of your language models. These metrics can help you identify areas where your model needs improvement and provide ins

In [67]:
agent_executor.invoke({"input": "what is wikipedia about ?"})



> Entering new AgentExecutor chain...
Wikipedia is a free online encyclopedia that allows anyone with an internet connection to access and contribute to its vast repository of knowledge. It was founded in 2001 by Jimmy Wales and Larry Sanger, and it has since become one of the most popular and widely-used websites in the world.

Wikipedia is a collaborative project that allows users to create and edit articles on a wide range of topics, from science and history to entertainment and culture. The site is maintained by a community of volunteer editors and contributors, who work together to ensure the accuracy and reliability of the information presented.

Wikipedia's content is available in hundreds of languages, and it is updated constantly as new information becomes available. The site is often used as a starting point for research, and it is also a valuable resource for people looking for quick answers to everyday questions.

Some of the key features of Wikipedia include:

1. Free ac

{'input': 'what is wikipedia about ?',
 'output': "Wikipedia is a free online encyclopedia that allows anyone with an internet connection to access and contribute to its vast repository of knowledge. It was founded in 2001 by Jimmy Wales and Larry Sanger, and it has since become one of the most popular and widely-used websites in the world.\n\nWikipedia is a collaborative project that allows users to create and edit articles on a wide range of topics, from science and history to entertainment and culture. The site is maintained by a community of volunteer editors and contributors, who work together to ensure the accuracy and reliability of the information presented.\n\nWikipedia's content is available in hundreds of languages, and it is updated constantly as new information becomes available. The site is often used as a starting point for research, and it is also a valuable resource for people looking for quick answers to everyday questions.\n\nSome of the key features of Wikipedia inc

Lets Include Memory Component to the Agents

In [70]:
message_history = ChatMessageHistory()

In [71]:
agent_with_chat_history = RunnableWithMessageHistory(
    agent_executor,
    # This is needed because in most real world scenarios, a session id is needed
    # It isn't really used here because we are using a simple in memory ChatMessageHistory
    lambda session_id: message_history,
    input_messages_key="input",
    history_messages_key="chat_history",
)

In [72]:
agent_with_chat_history.invoke(
    {"input": "hi! I am Asheesh"},
    # This is needed because in most real world scenarios, a session id is needed
    # It isn't really used here because we are using a simple in memory ChatMessageHistory
    config={"configurable": {"session_id": "<foo>"}},
)



> Entering new AgentExecutor chain...
Hi Asheesh! It's nice to meet you! I'm here to help you with any questions or tasks you may have. What's on your mind today? Do you need assistance with something specific or just want to chat?

> Finished chain.


{'input': 'hi! I am Asheesh',
 'chat_history': [],
 'output': "Hi Asheesh! It's nice to meet you! I'm here to help you with any questions or tasks you may have. What's on your mind today? Do you need assistance with something specific or just want to chat?"}

In [73]:
agent_with_chat_history.invoke(
    {"input": "can you explain me about Langchain briefly?"},
    # This is needed because in most real world scenarios, a session id is needed
    # It isn't really used here because we are using a simple in memory ChatMessageHistory
    config={"configurable": {"session_id": "<foo>"}},
)



> Entering new AgentExecutor chain...
LangChain is a fascinating topic!

LangChain is a cutting-edge AI technology that enables the creation of large-scale, high-quality language models. It's a type of transformer-based language model that's designed to generate human-like text, dialogue, and even entire stories.

Here's a brief overview:

1. **Architecture:** LangChain is built using a transformer architecture, which is a type of neural network that's particularly well-suited for natural language processing tasks.
2. **Training:** LangChain is trained on massive amounts of text data, which allows it to learn patterns, relationships, and nuances of language.
3. **Capabilities:** LangChain can generate text, respond to prompts, and even engage in conversations. It can also be fine-tuned for specific tasks, such as language translation, text summarization, and more.
4. **Advantages:** LangChain has several advantages over other language models, including its ability to generate coheren

{'input': 'can you explain me about Langchain briefly?',
 'chat_history': [HumanMessage(content='hi! I am Asheesh'),
  AIMessage(content="Hi Asheesh! It's nice to meet you! I'm here to help you with any questions or tasks you may have. What's on your mind today? Do you need assistance with something specific or just want to chat?")],
 'output': "LangChain is a fascinating topic!\n\nLangChain is a cutting-edge AI technology that enables the creation of large-scale, high-quality language models. It's a type of transformer-based language model that's designed to generate human-like text, dialogue, and even entire stories.\n\nHere's a brief overview:\n\n1. **Architecture:** LangChain is built using a transformer architecture, which is a type of neural network that's particularly well-suited for natural language processing tasks.\n2. **Training:** LangChain is trained on massive amounts of text data, which allows it to learn patterns, relationships, and nuances of language.\n3. **Capabiliti

In [74]:
agent_with_chat_history.invoke(
    {"input": "what is my name ?"},
    # This is needed because in most real world scenarios, a session id is needed
    # It isn't really used here because we are using a simple in memory ChatMessageHistory
    config={"configurable": {"session_id": "<foo>"}},
)

#here because of the memory function it remembers the provided name correctly



> Entering new AgentExecutor chain...
I remember! Your name is Asheesh!

> Finished chain.


{'input': 'what is my name ?',
 'chat_history': [HumanMessage(content='hi! I am Asheesh'),
  AIMessage(content="Hi Asheesh! It's nice to meet you! I'm here to help you with any questions or tasks you may have. What's on your mind today? Do you need assistance with something specific or just want to chat?"),
  HumanMessage(content='can you explain me about Langchain briefly?'),
  AIMessage(content="LangChain is a fascinating topic!\n\nLangChain is a cutting-edge AI technology that enables the creation of large-scale, high-quality language models. It's a type of transformer-based language model that's designed to generate human-like text, dialogue, and even entire stories.\n\nHere's a brief overview:\n\n1. **Architecture:** LangChain is built using a transformer architecture, which is a type of neural network that's particularly well-suited for natural language processing tasks.\n2. **Training:** LangChain is trained on massive amounts of text data, which allows it to learn patterns, rel